In [2]:
import torch
import os
import argparse
from torch.utils.data import Dataset, DataLoader
import torchtext
from collections import Counter
import numpy as np
import pandas as pd
import pickle

In [3]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [4]:
%cd gdrive/MyDrive/NLP-Project/GCDC_Corpus_v2/GCDC_rerelease/
%ls

/content/gdrive/MyDrive/NLP-Project/GCDC_Corpus_v2/GCDC_rerelease
Clinton_test.csv                  README.txt
Clinton_train.csv                 Yahoo_test.csv
Enron_test.csv                    Yahoo_train.csv
Enron_train.csv                   Yelp_test.csv
inverse_mapperYelp_test.csv.pkl   Yelp_test.csv.pkl
inverse_mapperYelp_train.csv.pkl  Yelp_train.csv
mapped_tokens_Yelp_test.csv.pkl   Yelp_train.csv.pkl
mapped_tokens_Yelp_train.csv.pkl


In [5]:
class Tokenizer:
    def __init__(self, file, threshold=5):
        self.file = file
        self.data = pd.read_csv(file)
        self.threshold = threshold

    def preprocess(self):
        tokenizer = torchtext.data.utils.get_tokenizer('spacy', language='en')
        tokens = []
        for text in self.data['text'].tolist():
            tokens.append(tokenizer(text))
        counter = Counter()
        for line in tokens:
            for word in line:
                counter[word] += 1
        # print(len(counter.items()), len(counter.most_common()))

        # remove all words that have frequency less than threshold
        # counter_threshold = {k:v for k,v in counter.items() if v >= self.threshold}

        # create mappings
        # mapper = {word:idx+1 for idx,word in enumerate(counter_threshold.keys())}
        # inverse_mapper = {idx+1:word for idx,word in enumerate(counter_threshold.keys())}

        # sos_idx = len(counter_threshold.keys())
        # eos_idx = len(counter_threshold.keys()) + 1
        # other_idx = len(counter_threshold.keys()) + 2

        # mapped_tokens = []

        # for line in tokens:
        #     mapped_line = [sos_idx]
        #     for word in line:
        #       # map words to their mappings and to other otherwise
        #         mapped_line.append(mapper.get(word, other_idx))
        #     mapped_line.append(eos_idx)
        #     mapped_tokens.append(mapped_line)

        # inverse_mapper[other_idx] = "__OTHER__"
        # inverse_mapper[sos_idx] = "__SOS__"
        # inverse_mapper[eos_idx] = "__EOS__"
        # inverse_mapper[0] = "__PADDING__"

        mapper = {word[0]: idx+1 for idx,
                  word in enumerate(counter.most_common())}
        inverse_mapper = {idx+1: word[0] for idx,
                          word in enumerate(counter.most_common())}

        # sos_idx = len(counter_threshold.keys())
        # eos_idx = len(counter_threshold.keys()) + 1
        other_idx = len(counter.keys())

        mapped_tokens = []

        for line in tokens:
            mapped_line = []
            for word in line:
              # map words to their mappings and to other otherwise
                mapped_line.append(mapper.get(word, other_idx))
            mapped_tokens.append(mapped_line)

        # inverse_mapper[other_idx] = "__OTHER__"
        # inverse_mapper[sos_idx] = "__SOS__"
        # inverse_mapper[eos_idx] = "__EOS__"
        # inverse_mapper[0] = "__PADDING__"

        return mapped_tokens, inverse_mapper

In [6]:
train = Tokenizer("Yelp_train.csv")
test = Tokenizer("Yelp_test.csv")

In [7]:
train.data


,text_id,subject,text,ratingA1,ratingA2,ratingA3,labelA,ratingM1,ratingM2,ratingM3,ratingM4,ratingM5,labelM
0,FUYQ99EUHg2TOHMMTy7cFQ,Seasons Buffet,Most months this buffet at the Silverton has o...,3,2,3,3,3,2,3,2,1,2
1,8Mkhxopb8jzL748V-oCVKg,The Cleaning Authority - Phoenix,"Awful, I should have looked at yelp. I receiv...",2,3,2,3,2,3,2,3,2,3
2,db7i0JVq9s_AKVvOoHBNxQ,Bad Owl Coffee,"I don't know how I found this place on Yelp, b...",2,3,3,3,3,3,2,2,3,3
3,9HebYLRstsuqti5_7P0hEg,Moe's Southwest Grill,"I love Moe's. So much better than Chipotle, IM...",2,3,3,3,2,3,3,2,3,3
4,7KgfshQrZ9O9iMIABtm59Q,Nirvana the Flavors of India,"Visiting from the US, I was looking for a some...",3,2,2,3,2,3,3,1,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,9ISZhCjisjmIpINkfVDkHg,Target,I always go to this target since I live down t...,3,1,1,1,2,2,1,2,3,2
996,-grSvaH5qWD3yn4NiLT3tg,Malones's Bakeries,Malones is a bakers located on Slateford Road ...,1,2,2,1,2,2,2,1,2,1
997,RF7zy-XMzDsUaCa5zq33Vg,Ernie's Restaurant & Bar,Seven of us went to Ernie's for a Christmas ka...,2,3,2,3,2,1,2,3,2,2
998,KiLwmdDe_PDPdov8coneBA,Urban Taco,Omg the food here is sooooooo delish! The mea...,2,1,2,1,2,3,2,2,3,3


In [8]:
# lst=[]
# for i in range(1000):
#   if train.data['labelA'][i]==3:
#     lst.append([0,0,1])
#   elif train.data['labelA'][i]==2:
#     lst.append([0,1,0])
#   elif train.data['labelA'][i]==1:
#     lst.append([1,0,0])

# train.data['h_e']=lst

from numpy import array
from numpy import argmax
from tensorflow.keras.utils import to_categorical

lst = array(train.data['labelA'])
encoded = to_categorical(lst)
print(encoded)
# inverted = argmax(encoded[0])
# print(inverted)

# train.data['h_e'] = encoded

[[0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 ...
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]]


In [9]:
# lst=[]
# for i in range(200):
#   if test.data['labelA'][i]==3:
#     lst.append([0,0,1])
#   elif test.data['labelA'][i]==2:
#     lst.append([0,1,0])
#   elif test.data['labelA'][i]==1:
#     lst.append([1,0,0])

# test.data['h_e']=lst

lst = array(test.data['labelA'])
t_encoded = to_categorical(lst)
# test.data['h_e'] = encoded

In [10]:
train_mapping, inv_train_mapping = train.preprocess()
test_mapping, inv_test_mapping = test.preprocess()

In [11]:
# new_data = open('/content/drive/MyDrive/GCDC_rerelease/mapped_tokens_Yelp_train.csv.pkl','rb')
# new_t_data = open('/content/drive/MyDrive/GCDC_rerelease/mapped_tokens_Yelp_test.csv.pkl','rb')
# train_mapping = pickle.load(new_data)
# test_mapping = pickle.load(new_t_data)

len(train_mapping)
# train_mapping
train.data['encoding'] = train_mapping
test.data['encoding'] = test_mapping
train.data

,text_id,subject,text,ratingA1,ratingA2,ratingA3,labelA,ratingM1,ratingM2,ratingM3,ratingM4,ratingM5,labelM,encoding
0,FUYQ99EUHg2TOHMMTy7cFQ,Seasons Buffet,Most months this buffet at the Silverton has o...,3,2,3,3,3,2,3,2,1,2,"[1098, 481, 28, 482, 34, 2, 6264, 104, 68, 150..."
1,8Mkhxopb8jzL748V-oCVKg,The Cleaning Authority - Phoenix,"Awful, I should have looked at yelp. I receiv...",2,3,2,3,2,3,2,3,2,3,"[4373, 3, 5, 224, 27, 354, 34, 1722, 1, 9, 5, ..."
2,db7i0JVq9s_AKVvOoHBNxQ,Bad Owl Coffee,"I don't know how I found this place on Yelp, b...",2,3,3,3,3,3,2,2,3,3,"[5, 50, 23, 144, 136, 5, 276, 28, 39, 24, 755,..."
3,9HebYLRstsuqti5_7P0hEg,Moe's Southwest Grill,"I love Moe's. So much better than Chipotle, IM...",2,3,3,3,2,3,3,2,3,3,"[5, 201, 2432, 26, 1, 173, 120, 166, 108, 2125..."
4,7KgfshQrZ9O9iMIABtm59Q,Nirvana the Flavors of India,"Visiting from the US, I was looking for a some...",3,2,2,3,2,3,3,1,2,2,"[4383, 60, 2, 1897, 3, 5, 8, 205, 12, 6, 1729,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,9ISZhCjisjmIpINkfVDkHg,Target,I always go to this target since I live down t...,3,1,1,1,2,2,1,2,3,2,"[5, 131, 69, 7, 28, 4361, 213, 5, 484, 160, 2,..."
996,-grSvaH5qWD3yn4NiLT3tg,Malones's Bakeries,Malones is a bakers located on Slateford Road ...,1,2,2,1,2,2,2,1,2,1,"[4367, 13, 6, 12312, 713, 24, 12313, 12314, 14..."
997,RF7zy-XMzDsUaCa5zq33Vg,Ernie's Restaurant & Bar,Seven of us went to Ernie's for a Christmas ka...,2,3,2,3,2,1,2,3,2,2,"[12317, 10, 96, 126, 7, 12318, 26, 12, 6, 1433..."
998,KiLwmdDe_PDPdov8coneBA,Urban Taco,Omg the food here is sooooooo delish! The mea...,2,1,2,1,2,3,2,2,3,3,"[12327, 2, 45, 55, 13, 12328, 2917, 15, 9, 17,..."


In [12]:
train.data

,text_id,subject,text,ratingA1,ratingA2,ratingA3,labelA,ratingM1,ratingM2,ratingM3,ratingM4,ratingM5,labelM,encoding
0,FUYQ99EUHg2TOHMMTy7cFQ,Seasons Buffet,Most months this buffet at the Silverton has o...,3,2,3,3,3,2,3,2,1,2,"[1098, 481, 28, 482, 34, 2, 6264, 104, 68, 150..."
1,8Mkhxopb8jzL748V-oCVKg,The Cleaning Authority - Phoenix,"Awful, I should have looked at yelp. I receiv...",2,3,2,3,2,3,2,3,2,3,"[4373, 3, 5, 224, 27, 354, 34, 1722, 1, 9, 5, ..."
2,db7i0JVq9s_AKVvOoHBNxQ,Bad Owl Coffee,"I don't know how I found this place on Yelp, b...",2,3,3,3,3,3,2,2,3,3,"[5, 50, 23, 144, 136, 5, 276, 28, 39, 24, 755,..."
3,9HebYLRstsuqti5_7P0hEg,Moe's Southwest Grill,"I love Moe's. So much better than Chipotle, IM...",2,3,3,3,2,3,3,2,3,3,"[5, 201, 2432, 26, 1, 173, 120, 166, 108, 2125..."
4,7KgfshQrZ9O9iMIABtm59Q,Nirvana the Flavors of India,"Visiting from the US, I was looking for a some...",3,2,2,3,2,3,3,1,2,2,"[4383, 60, 2, 1897, 3, 5, 8, 205, 12, 6, 1729,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,9ISZhCjisjmIpINkfVDkHg,Target,I always go to this target since I live down t...,3,1,1,1,2,2,1,2,3,2,"[5, 131, 69, 7, 28, 4361, 213, 5, 484, 160, 2,..."
996,-grSvaH5qWD3yn4NiLT3tg,Malones's Bakeries,Malones is a bakers located on Slateford Road ...,1,2,2,1,2,2,2,1,2,1,"[4367, 13, 6, 12312, 713, 24, 12313, 12314, 14..."
997,RF7zy-XMzDsUaCa5zq33Vg,Ernie's Restaurant & Bar,Seven of us went to Ernie's for a Christmas ka...,2,3,2,3,2,1,2,3,2,2,"[12317, 10, 96, 126, 7, 12318, 26, 12, 6, 1433..."
998,KiLwmdDe_PDPdov8coneBA,Urban Taco,Omg the food here is sooooooo delish! The mea...,2,1,2,1,2,3,2,2,3,3,"[12327, 2, 45, 55, 13, 12328, 2917, 15, 9, 17,..."


In [13]:
test.data

,text_id,subject,text,ratingA1,ratingA2,ratingA3,labelA,ratingM1,ratingM2,ratingM3,ratingM4,ratingM5,labelM,encoding
0,40fxk9sDmyhh7S6V6Uuimg,Tacos Sahuaro,Tough little place to find if you haven't been...,2,2,1,1,3,2,1,2,3,2,"[2216, 115, 35, 7, 173, 72, 22, 26, 19, 81, 44..."
1,zmFn_KhpZPFsvz6Zp9_ogw,GRAZ Kitchen Fresh,I love this place. Like seriously crave their ...,1,2,1,1,3,3,3,2,3,3,"[5, 119, 27, 35, 1, 881, 562, 1453, 67, 40, 79..."
2,zVU8PV5i9mKTJtBNIv_2Sg,Jalapeño Buck's,"No indoor seating but there was shade, misters...",2,1,2,1,1,2,2,2,1,1,"[460, 1091, 391, 21, 44, 8, 2230, 3, 1456, 3, ..."
3,bc0dLKd-9LYrRK9GmmWmXg,Redfin Blues,This restaurant is a waterfront spot that serv...,3,3,2,3,3,2,2,3,2,3,"[89, 113, 13, 6, 2233, 234, 16, 2234, 2235, 10..."
4,4DrOVgI9rdBFb67kZ26miw,Humble,Absolutely Delicious! Last week I had lemon la...,3,2,2,3,3,1,2,2,3,2,"[1466, 1467, 15, 1468, 363, 5, 31, 1105, 1106,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,Ld1-4STmN0Qe-xVyX5N8Iw,In-N-Out Burger,Let's face it! Most every burger aficionado in...,2,2,3,3,1,3,3,2,3,3,"[1373, 29, 642, 10, 15, 978, 241, 165, 4918, 4..."
196,XZbkHGhOo_Pu-ypw4j_bvw,Rent-A-Wheel,Filled out an application online got a call fr...,1,3,2,2,2,2,1,3,2,2,"[4955, 45, 71, 4956, 2205, 87, 6, 411, 64, 144..."
197,sB-Lx1QF2hI8yQpb_SdINA,Executive Hotel Cosmopolitan,Stayed here over the Christmas holidays and se...,2,2,3,3,3,2,2,2,3,3,"[4971, 47, 129, 2, 4972, 1874, 4, 2044, 30, 6,..."
198,GifRhXPlxytoRkhyEWbqjQ,Chelsea's Kitchen,The patio is a great place to sit back and enj...,2,2,3,3,2,3,3,3,2,3,"[18, 622, 13, 6, 65, 35, 7, 623, 54, 4, 416, 2..."


In [14]:
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

In [15]:
np.random.seed(7)
X_train = sequence.pad_sequences(train.data['encoding'],maxlen = 500)
y_train = encoded#train.data['h_e']
X_test = sequence.pad_sequences(test.data['encoding'],maxlen=500)
y_test = t_encoded#test.data['h_e']





In [16]:
embedding_vector_length = 32
model = Sequential()
model.add(Embedding(30000,embedding_vector_length,input_length = 500))
model.add(LSTM(4))
model.add(Dense(4,activation = 'softmax'))
model.compile(loss ='categorical_crossentropy', optimizer = 'adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_train,y_train, epochs = 40, batch_size=64)

scores = model.evaluate(X_test, y_test, verbose =0)
print("Accuracy: ",(scores[1]*100))

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 500, 32)           960000    
                                                                 
 lstm (LSTM)                 (None, 4)                 592       
                                                                 
 dense (Dense)               (None, 4)                 20        
                                                                 
Total params: 960,612
Trainable params: 960,612
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/40
16/16 [==============================] - 7s 72ms/step - loss: 1.3695 - accuracy: 0.4880
Epoch 2/40
16/16 [==============================] - 1s 71ms/step - loss: 1.3206 - accuracy: 0.5110
Epoch 3/40
16/16 [==============================] - 1s 71ms/step - loss: 1.2588 - accuracy: 0.5110
Epoch 4/40
1